<a href="https://colab.research.google.com/github/GuilhermeSotti/macine-learning/blob/main/TensorFlow_Code7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def main():
  train_dir = './train_images/'
  rock_dir = os.path.join(train_dir + 'rock')
  paper_dir = os.path.join(train_dir + 'paper')
  scissors_dir = os.path.join(train_dir + 'scissors')

  rock_list = os.listdir(rock_dir)
  paper_list = os.listdir(paper_dir)
  scissors_list = os.listdir(scissors_dir)

  print('total training rock images ', len(rock_dir), ' ', rock_list[:10])
  print('total training paper images ', len(paper_dir), ' ', paper_list[:10])
  print('total training scissor images ', len(scissors_dir), ' ', scissors_list[:10])

  pic_index = 2

  next_rock = [os.path.join(rock_dir, file_name)
                for file_name in rock_file[pic_index - 2: pic_index]]
  next_paper = [os.path.join(paper_dir, file_name)
                for file_name in paper_dir[pic_index - 2: pic_index]]
  next_scissors = [os.path.join(scissors_dir, file_name)
                for file_name in scissors_dir[pic_index - 2: pic_index]]

  for i, img_path in enumerate(next_rock + next_paper + next_scissors):
    print(img_path)
    img = mpimg.imread(img_path)
    plt.imshow(img)
    plt.axis('Off')
    plt.show()

if __name__ == '__main__':
  main()

In [22]:
from tensorflow.python.keras.preprocessing.image import load_img
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from keras_preprocessing import image
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import os

def main():
  TRAINING_DIR = "./train_images/"
  training_data_gen = ImageDataGenerator(
      rescale = 1./255,
      rotation_range = 40,
      width_shift_range = 0.2,
      height_shift_range = 0.2,
      shear_range = 0.2,
      zoom_range = 0.2,
      horizontal_flip = True,
      fill_mode ='nearest'
  )

  VALIDATION_DIR = "./test_image/"
  validation_data_gen = ImageDataGenerator(rescale = 1./255)
  
  train_generator = training_data_gen.flow_from_directory(
      TRAINING_DIR,
      target_size = (150, 150),
      class_mode = 'categorical',
      batch_size = 126
  )
  validation_generator = validation_data_gen.flow_from_directory(
      VALIDATION_DIR,
      target_size = (150, 150),
      class_mode = 'categorical',
      batch_size = 126
  )

  model = Sequential([
          layers.Conv2D(64, (3, 3), activation = 'relu', input_shape = (150, 150)),
          layers.MaxPool2D(2, 2),
          layers.Conv2D(64, (3, 3), activation = 'relu'),
          layers.MaxPool2D(2, 2),
          layers.Conv2D(128, (3, 3), activation = 'relu'),
          layers.MaxPool2D(2, 2),
          layers.Conv2D(128, (3, 3), activation = 'relu'),
          layers.MaxPool2D(2, 2),

          layers.Flatten(),
          layers.Dropout(0.5),

          layers.Dense(512, activation = 'relu'),
          layers.Dense(3, activation = 'sotfmax')            
  ])

  model.sumary()
  model.compile(
      loss = 'categorial_crossentropy',
      optimizer = 'rmsprop', 
      metrics = ['accuracy']
  )
  history = model.fit(
      train_generator, 
      epochs = 25,
      steps_per_epoch = 20,
      validation_data = validation_generator,
      verbose = 1
  )
  model.save('rmp.h5')

  files = [os.path.join(dp, f)
          for dp, dn, filename in os.walk(VALIDATION_DIR)
          for f in filename
          if os.path.splitext(f) == '.png']
  print(files)

  rock = 0
  paper = 0
  scissor = 0

  for fn in files:
    img = load_img(fn, target_size = (150, 150))
    y = image.img_to_array(img)
    x = np.expand_dims(y, axis = 0)
    images = np.vstack([x])
    classes = model.predict(images, batch_size = 10)
    if classes[0][0] > classes[0][1] and classes[0][0] > classes[0][2]:
      print(fn + " is a paper")
      paper += 1
    elif classes[0][1] > classes[0][0] and classes[0][1] > classes[0][2]:
      print(fn + " is a rock")
      rock += 1
    else:
      print(fn + " is a scissor")
      scissor += 1

  print('TOTAL PAPER: ', paper)
  print('TOTAL ROCK: ', rock)
  print('TOTAL SCISSOR: ', scissor)

  acc = history.history(['accuracy'])
  val_acc = history.history['val_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  epochs = range(len(acc))

  plt.tight_layout()
  plt.show()
  plt.title("Prediction of rock, paper or scissor")
  plt.bar(['paper (0)'], paper, color='b')
  plt.bar(['rock (1)'], rock, color='r')
  plt.bar(['scissor (2)'], scissor, color='g')

  plt.plot(epochs, acc, 'r', label = 'Traning accuracy')
  plt.plot(epochs, val_acc, 'b', label = 'Validation accuracy')
  plt.title('Training and Validation accuracy')
  plt.legend()
  plt.show()

if __name__ == '__main__':
  main()

FileNotFoundError: ignored